# Unsupervised Learning - Unsupervised Image Clustering comparison with PCA, NMF, Autoencoder and DEC
by M. Giordano

## Introduction

This is an Unsupervised Learning task focused on Image Clustering comparison with different unsupervised models.<br>
The dataset in composed of various photorealistis natural AI generated images .<br>
Our goal is to build a series of models to compare their ability to correctly clusterize into three groups: animals, birds and landscapes.

This will be achieved through EDA, data cleaning, feature engineering, and model development from scratch.


This work covers all the required points of the rubric, following the steps in the same order of appearance:

1. Gather data, determine the method of data collection and provenance of the data (3 points)

2. Identify an Unsupervised Learning Problem (6 points)

3. Exploratory Data Analysis (EDA) — Inspect, Visualize, and Clean the Data (26 points)

4. Perform Analysis Using Unsupervised Learning Models of your Choice, Present Discussion, and Conclusions (70 points)

5. Produce Deliverables: High-Quality, Organized Jupyter Notebook Report, Video Presentation, and GitHub Repository (35 points)


<br>
This notebook, and the pdf version are available at the following Github repository:

https://github.com/michele-giordano/unsupervised_learning_final_assignment/

The video presentation can be seen at the following link:

https://www.youtube.com/watch?v=fvw-nV2-CKw

## Step 1: Gather data, determine the method of data collection and provenance of the data

### 1.1 Data Collection and Provenance

The dataset used in this work was created directly by the author, taking initial inspiration from image clustering competitions found on Kaggle.<br>
During the review phase, I noticed that the datasets available there were generally of poor visual quality and highly inconsistent, often containing noisy, low-resolution, or poorly framed images.<br>
These datasets reached a maximum accuracy values around 0.8 even under supervised learning conditions, indicating that the visual variability present was not informative but mostly chaotic.

For this reason, they were not considered suitable for an unsupervised learning task where the structure of the data itself must guide the formation of meaningful clusters.

To address this limitation, Stable Diffusion was used to generate a new dataset with a similar photographic style, but with controlled visual quality.<br>
The images retain natural and realistic appearance, and include subjects that are not trivially separable, yet are clearly represented. This approach ensures a dataset that is coherent, reproducible, and appropriate for evaluating unsupervised image clustering methods.

### 1.2 Method of Data Collection

The images were generated with the explicit goal of representing the subjects in a structured and classifiable manner, while maintaining a high degree of heterogeneity within the dataset. More than five hundred images were created from scratch, including a variety of animal species, different types of birds, and diverse landscape scenarios across multiple seasons and lighting conditions. The intent was to build a dataset that is visually coherent but not uniform, allowing for meaningful variation along relevant visual features.

Considering that the unsupervised methods used in this work will attempt to form clusters directly from pixel-level information, several images were deliberately designed to be challenging from a clustering perspective. Examples include polar bears against snowy backgrounds or brown-fur animals placed in forest environments, , where subject and background share very similar color distributions and silhouette is less distinct. 

These controlled difficulties were introduced to prevent the dataset from being trivially separable and to ensure that the clustering methods must effectively extract and leverage visual structure rather than relying on simple color or texture cues.

## Step 2: Identify the Unsupervised Learning Problem

<b>This is an unsupervised task of image clustering based on photorealistic images</b>.<br>
The goal is to conduct EDA, perform data cleaning, extract meaningful visual features, and train a clustering model able to assign images to one of three conceptual groups: 
- animals, 
- birds, 
- landscapes. 

The model does not receive labeled data during training; instead, the structure must emerge from the visual patterns present in the images themselves.
The evaluation therefore focuses on whether the resulting clusters form coherent visual groupings that align with these three categories, without relying on supervision or ground-truth labels.

<b>This work goes through all the rubric requirements, implementing the necessary code entirely from scratch.</b><br>
The analysis is designed to independently explore different approaches to feature extraction, dimensionality reduction, and clustering, rather than reproducing any existing kernels that could be found on Kaggle on similar tasks. 

Each stage of the pipeline, from preprocessing to model comparison, is developed and validated within this notebook to ensure methodological originality and full adherence to the rubric's expectations for analytical depth and reproducibility.

## Step 3: Exploratory Data Analysis (EDA) - Inspect, Visualize, and Clean the Data

### 3.1 Describe the factors or components that make up the dataset 

As a first operation, we inspect the size and shape of the dataset to understand the nature of the collected data.<br>
This step is essential to verify data integrity, identify potential irregularities, and gain an initial intuition about the structure of the images.

In [ ]:
# imports
import os
import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, homogeneity_score, completeness_score, v_measure_score, silhouette_score, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# --- Exploratory Data Analysis (EDA) for RGB dataset ---

UNLABELLED_DATASET = np.load(f"data/dataset_256x256_unlabeled_shuffled.npz", allow_pickle=True)
LABELLED_DATASET = np.load(f"data/dataset_256x256_labeled_shuffled.npz", allow_pickle=True)

data = UNLABELLED_DATASET
images = data["X"]
X_raw = data["X"]

print("Total number of images:", len(X_raw))

widths = []
heights = []

for img in X_raw:
    if img is None or img.ndim != 3:
        continue
    h, w = img.shape[:2]
    heights.append(h)
    widths.append(w)

heights = np.array(heights)
widths = np.array(widths)

print("Minimum dimension:", heights.min(), "x", widths.min())
print("Maximum dimension:", heights.max(), "x", widths.max())

print("Number of distinct resolutions:", len(set(zip(heights, widths))))

# --- Visualize batches of RGB images ---
def show_batch(images, start_index=0, batch_size=100):
    end_index = min(start_index + batch_size, images.shape[0])
    subset = images[start_index:end_index]

    cols = 16
    rows = int(np.ceil(batch_size / cols))
    rows = cols
    plt.figure(figsize=(12, 12))
    for i, img in enumerate(subset):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img.astype(np.uint8))
        plt.axis("off")
    plt.suptitle(f"Images {start_index}–{end_index-1}", fontsize=14)
    plt.tight_layout()
    plt.show()

# Example: view first batch
show_batch(X_raw, 0, 100)

![EDA](images/eda.png)


We can see that the dataset consists of 599 images with resolutions that range from  44 x 108 to 256 x 256.

This diversity is intentional, as it focuses the attention on doing a proper data cleaning. Since the dataset is declared having 256x256, all other dimensions are considered noise to be removed. 

Each image can be represented as a three-dimensional matrix of pixel values, where the two spatial dimensions correspond to height and width, and the third dimension corresponds to the RGB color channels.

These pixel values constitute the fundamental factors of the dataset, as all subsequent analysis steps, including feature extraction, dimensionality reduction, and clustering,  will be based on the relationships and variations within these numerical components.
No additional metadata or external features are provided, ensuring that all learning emerges directly from the visual information contained in the pixel matrices.

The graphical visualization of the first 100 records shows that this dataset is a collection of different natural photorealistic images, including birds, cats, dogs, and a variety of landscapes with different illumination and depicting diverse seasons.

### 3.2 Data Cleaning

Firstly, we will remove all the images with resolution different than 256x256, then we will search for duplicates and remove them as well.

![Data Cleaning](images/cleaned_dataset.png)


After the data cleaning phase, the dataset consists of 500 images, each standardized to a 256 × 256 resolution.
The analysis of pixel intensities shows values ranging from 0 to 255, with a global mean of  123.635 and a standard deviation of 73.127.

From the histograms of the RGB channels and the global pixel distribution, we can observe a balanced and well-spread color intensity across the three channels, with no saturation or clipping at the extremes.
The distributions reveal a natural variability typical of photorealistic images, confirming that the dataset covers both bright and dark regions without dominant bias toward any single color component.
This suggests that the dataset is visually diverse, properly normalized, and suitable for feature extraction and clustering in the following stages.


### 3.3 Describe correlations between different factors of the dataset and justify your assumption that they are correlated or not correlated.

In traditional tabular datasets, correlation analysis is typically used to detect redundant features that can be removed because they are linear combinations of others.

In this case, however, <b>each feature corresponds to a fixed pixel position within a 256×256×3 image matrix.</b>
The relationships among these features are primarily spatial rather than statistical: pixel values are not independent variables but elements of structured visual compositions. Adjacent pixels often share similar values due to color continuity within objects, while distant pixels may vary significantly depending on the spatial layout and content of the image.

For this reason, a standard correlation matrix between pixel intensities would not yield meaningful insights about feature redundancy or independence. The observed dependencies are intrinsic to the geometry of the images rather than to the data distribution itself.

Therefore, direct correlation analysis is not applied in this context. Instead, potential dependencies among features are implicitly captured and reduced through dimensionality reduction techniques such as PCA, NMF, or autoencoders, which are designed to extract compact latent representations that preserve the most relevant visual structure of the dataset.


### 3.4 Determine if any data needs to be transformed.

Since each record already consists of numerical pixel values, no mandatory transformations are required to make the dataset suitable for analysis.

Nevertheless, certain transformations of the pixel values may be beneficial depending on the modeling approach adopted.

These adjustments can improve numerical consistency and ensure that the representation of the images remains balanced across all features, allowing subsequent algorithms to operate on a coherent and comparable data scale.


### 3.5 Using your hypothesis, indicate if it's likely that you should transform data

Given that the dataset consists of RGB pixel matrices, no single transformation is universally required in advance.

Normalization or standardization should be applied only when justified by the specific clustering method being employed, since different algorithms react differently to the scale of the input features.

For distance-based methods such as PCA, KMeans, or linkage clustering, scaling procedures can be introduced to maintain numerical coherence and prevent individual channels or intensity ranges from dominating the computation. 

In contrast, models that operate directly on raw pixel structures or deep-learning-based feature extractors may not require preliminary transformations.

Where appropriate, additional adjustments such as contrast correction, saturation control, or resolution changes may be introduced later to refine the consistency of the visual data and improve the quality of the extracted features.

### 3.6 You should determine if your data has outliers or needs to be cleaned in any way.

As shown in the preliminary integrity checks performed in section 3.5, the dataset contains no missing, invalid, or inconsistent values.<br>All images share the same shape and data type, and no duplicated or corrupted records are present.

Given that each record represents a real image, pixel variations cannot be interpreted as statistical outliers but rather as genuine visual differences among samples.

Therefore, no data removal, interpolation, or substitution has been applied. The dataset can be considered clean and ready for use in the subsequent dimensionality reduction and clustering steps.

### 3.7 If you believe that specific factors will be more important than others in your analysis, you should mention which and why. You will use this to confirm your intuitions in your final write-up.

In this dataset, the most informative factors are not individual pixels but patterns of pixel intensities that describe shapes and textures within the images. 

These latent structures will be identified through matrix decomposition techniques such as PCA and NMF, which extract the underlying components that best represent visual variability. 

Their comparison will be performed in the next section to evaluate which representation provides more meaningful features for clustering.


### 3.8 Balancing the dataset

Considering that unsupervised models benefit from a balanced dataset, the three classes were equalized in size. 

The reduction in total samples is a reasonable trade-off, as methods like NMF, autoencoders, and DEC gain stability and clearer cluster structure when the class distribution is even, while PCA is the only method that remains largely unaffected.


## Step 4: Perform Analysis Using Unsupervised Learning Models of Your Choice, Present Discussion, and Conclusions

In this section, we build and compare multiple models to analyze how different unsupervised learning techniques perform when applied to the photorealistic dataset created from scratch.

The objective is to understand how each approach extracts latent features, organizes visual information, and forms clusters that reflect meaningful semantic structures within the images.

While the focus of this work remains on unsupervised learning, a simple supervised baseline is also included to provide a reference point and to highlight the practical gap between self-organized and label-driven methods.

The following models are considered:

* **PCA + K-Means:** a linear projection combined with distance-based clustering, serving as a baseline for variance-driven separation.
* **Hierarchical Linkage + NMF:** a hybrid approach that combines non-negative matrix factorization with hierarchical clustering to identify additive and interpretable visual components.
* **Autoencoder:** a neural model trained to reconstruct input images through a compact latent representation, capturing non-linear dependencies among pixels.
* **Deep Embedded Clustering (DEC):** an architecture that jointly optimizes representation learning and clustering to improve latent separability.
* **DEC on Grayscale Images:** a variant used to assess the role of color information in cluster formation by training the DEC model on luminance-only inputs.
* **Supervised Semantic Classifier (baseline):** a reference model trained on a small labeled subset to estimate the achievable upper-bound accuracy and to contextualize the limitations of unsupervised approaches.

This multi-model comparison allows both quantitative and qualitative evaluation of clustering behavior, illustrating how increasing representational power affects the emergence of semantic structure.
At the same time, the supervised baseline serves to confirm the internal coherence and overall quality of the dataset itself, showing that the lower accuracy of unsupervised methods reflects the intrinsic challenge of discovering structure without labels rather than any limitation of the data.


### Metrics

In this report, clustering quality is evaluated using the following metrics, each capturing a different aspect of agreement between the predicted clusters and the ground-truth categories in the labelled_dataset.

- ARI: Adjusted Rand Index measures the similarity between two partitions while correcting for chance. Higher values indicate stronger alignment between clusters and true labels.
- NMI: Normalized Mutual Information quantifies how much information about the true labels is preserved by the clustering, scaled between 0 and 1 for comparability.
- Hom: Homogeneity measures whether each cluster contains samples belonging to a single true class, penalizing mixed-class clusters.
- Complt: Completeness evaluates whether all samples of a given true class are assigned to the same cluster, penalizing fragmentation.
- V-Measure: The harmonic mean of homogeneity and completeness, providing a balanced summary of cluster purity and class cohesion.
- Opt. Accuracy: Optimized accuracy aligns cluster IDs with true labels using the Hungarian algorithm, estimating the best possible accuracy achievable after resolving the arbitrary label permutations inherent in unsupervised clustering.

In addition to the main metrics, we also examined the confusion matrix after applying the Hungarian alignment. This representation, however, is not always meaningful in unsupervised settings, especially when the learned clusters do not mirror the ground-truth distribution. 

For this reason, the evaluation focuses on permutation-invariant metrics described above, which provide a more reliable indication of the underlying partition quality.


### 4.1 PCA and K-Means


The first model applies Principal Component Analysis (PCA) to reduce each image to a compact set of components that retain most of the variance, summarizing global color and texture while removing redundancy. 

On this representation, we apply K-Means to group images with similar latent features. Because centroid-based methods adapt well to the geometry of PCA-transformed data, K-Means is generally expected to perform better on images, especially when the main structures are captured by the leading components.

As an alternative, we also apply hierarchical linkage clustering (Ward’s method) on the same PCA space. Unlike K-Means, linkage builds the cluster structure through irreversible merges that prioritize local variance minimization, offering a more rigid interpretation of the reduced feature space.

Using these two PCA-based variants allows us to compare how flexible centroid-driven optimization and hierarchical merging behave when given the same linear embedding of the images.


In [ ]:
# ============================================================
# Evaluate clustering vs ground truth (Category-based)
# ============================================================
def evaluate_clustering(labels_pred, labelled_dataset):
    """
    Evaluate clustering metrics against ground-truth categories.

    Args:
        labels_pred (array-like): predicted cluster labels
        labelled_dataset (dict or np.lib.npyio.NpzFile): must contain 'Category'
    """

    # ------------------------------------------------------------
    # Extract ground-truth categories
    # ------------------------------------------------------------
    if isinstance(labelled_dataset, dict):
        if "Category" in labelled_dataset:
            labels_true = labelled_dataset["Category"]
        elif "category" in labelled_dataset:
            labels_true = labelled_dataset["category"]
        else:
            raise ValueError("LABELLED_DATASET must contain a 'Category' or 'category' key.")
    else:
        # if it's a npz file
        if hasattr(labelled_dataset, "files") and "Category" in labelled_dataset.files:
            labels_true = labelled_dataset["Category"]
        elif hasattr(labelled_dataset, "files") and "category" in labelled_dataset.files:
            labels_true = labelled_dataset["category"]
        else:
            raise ValueError("LABELLED_DATASET must contain a 'Category' or 'category' key.")

    labels_true = np.array(labels_true)
    labels_pred = np.array(labels_pred)

    # safety check
    if len(labels_true) != len(labels_pred):
        min_len = min(len(labels_true), len(labels_pred))
        print(f"⚠️ Warning: length mismatch ({len(labels_true)} vs {len(labels_pred)}). Truncating to {min_len}.")
        labels_true = labels_true[:min_len]
        labels_pred = labels_pred[:min_len]

    # convert string categories to integer codes if necessary
    if labels_true.dtype.type is np.str_ or labels_true.dtype == object:
        unique_classes, encoded = np.unique(labels_true, return_inverse=True)
        labels_true = encoded

    # ------------------------------------------------------------
    # Metrics
    # ------------------------------------------------------------
    ari = adjusted_rand_score(labels_true, labels_pred)
    nmi = normalized_mutual_info_score(labels_true, labels_pred)
    hom = homogeneity_score(labels_true, labels_pred)
    comp = completeness_score(labels_true, labels_pred)
    v_meas = v_measure_score(labels_true, labels_pred)

    # ------------------------------------------------------------
    # Optimized accuracy (Hungarian)
    # ------------------------------------------------------------
    n_classes = max(labels_true.max(), labels_pred.max()) + 1
    contingency = np.zeros((n_classes, n_classes), dtype=np.int64)
    for t, p in zip(labels_true, labels_pred):
        contingency[t, p] += 1

    row_ind, col_ind = linear_sum_assignment(-contingency)
    optimized_acc = contingency[row_ind, col_ind].sum() / contingency.sum()

    # ------------------------------------------------------------
    # Print results
    # ------------------------------------------------------------
    print("\n=== Clustering Evaluation vs Ground Truth (Category) ===")
    print(f"Adjusted Rand Index (ARI):       {ari:.4f}")
    print(f"Normalized Mutual Information:   {nmi:.4f}")
    print(f"Homogeneity:                     {hom:.4f}")
    print(f"Completeness:                    {comp:.4f}")
    print(f"V-Measure:                       {v_meas:.4f}")
    print(f"\nOptimized Accuracy (Hungarian):  {optimized_acc:.4f}")

    return {
        "ARI": ari,
        "NMI": nmi,
        "Homogeneity": hom,
        "Completeness": comp,
        "V-Measure": v_meas,
        "Optimized Accuracy": optimized_acc
    }


In [ ]:
# ===========================
# Generic 2D visualization 
# ===========================


def plot_clusters_2d(X_2d, labels_pred, labels_true=None, model_name="Model"):
    """
    Display predicted clusters and true categories side by side from 2D features.

    Args:
        X_2d (np.ndarray): 2D feature array of shape (n_samples, 2)
        labels_pred (array-like): predicted cluster labels
        labels_true (array-like, optional): true category labels
        model_name (str): model name to show in plot titles
    """


    # Sanity checks
    if X_2d.shape[1] != 2:
        pca_vis = PCA(n_components=2, random_state=42)
        X_vis = pca_vis.fit_transform(X_2d)  # reduce to 2D just for visualaization
        X_2d = X_vis

    n_cols = 2 if labels_true is not None else 1
    fig, axes = plt.subplots(1, n_cols, figsize=(10, 4))
    if not isinstance(axes, np.ndarray):
        axes = [axes]

    # --- Predicted clusters ---
    ax = axes[0]
    sc1 = ax.scatter(X_2d[:, 0], X_2d[:, 1], c=labels_pred, cmap="viridis", s=8)
    ax.set_title(f"Predicted Clusters ({model_name})", fontsize=10)
    ax.set_xlabel("Component 1", fontsize=8)
    ax.set_ylabel("Component 2", fontsize=8)
    ax.tick_params(axis="both", labelsize=8)

    # --- True categories ---
    if labels_true is not None:
        enc = LabelEncoder()
        y_true_encoded = enc.fit_transform(labels_true)
        ax2 = axes[1]
        sc2 = ax2.scatter(X_2d[:, 0], X_2d[:, 1], c=y_true_encoded, cmap="viridis", s=8)
        ax2.set_title(f"True Categories", fontsize=10)
        ax2.set_xlabel("Component 1", fontsize=8)
        ax2.set_ylabel("Component 2", fontsize=8)
        ax2.tick_params(axis="both", labelsize=8)

    plt.tight_layout()
    plt.show()


In [ ]:
# ============================================================
# 4.1 PCA + K-Means
# ============================================================

# ------------------------------------------------------------
# 1. Flatten images and standardize
# ------------------------------------------------------------
X = images.reshape(len(images), -1).astype(np.float32)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ------------------------------------------------------------
# 2. Apply PCA for dimensionality reduction
# ------------------------------------------------------------
n_components = 100
pca = PCA(n_components=n_components, random_state=42)
X_pca = pca.fit_transform(X_scaled)

explained_var = np.sum(pca.explained_variance_ratio_) * 100
print(f"PCA reduced to {n_components} components explaining {explained_var:.2f}% of variance")

# ------------------------------------------------------------
# 3. K-Means clustering
# ------------------------------------------------------------
k = 3  # expected clusters: animals, birds, landscapes
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
labels_kmeans_pca = kmeans.fit_predict(X_pca)

# ------------------------------------------------------------
# 4. Evaluate clustering
# ------------------------------------------------------------
sil_score = silhouette_score(X_pca, labels_kmeans_pca)
print(f"Silhouette Score: {sil_score:.4f}")

PCA reduced to 200 components explaining 95.73% of variance
Silhouette Score: 0.1785


![PCA](images/pca.png) 


#### PCA: K-means vs Linkage Comparison Results

The comparison of the evaluation metrics for both methods is summarised below:

| Metric                              | PCA K-means | PCA Linkage (Ward) |
| ----------------------------------- | ----------- | ------------------ |
| Adjusted Rand Index (ARI)           | 0.490       | 0.525              |
| Normalized Mutual Information (NMI) | 0.487       | 0.539              |
| Homogeneity                         | 0.478       | 0.529              |
| Completeness                        | 0.496       | 0.549              |
| V-Measure                           | 0.487       | 0.539              |
| Optimized Accuracy (Hungarian)      | 0.795       | 0.798              |


As we can see and as expected, K-means adapts well to the geometry of the PCA-transformed space through its centroid-based optimisation. It captures the dominant variance directions and maintains a stable separation across the three categories. Hierarchical linkage behaves differently, but when the class densities are more uniform it benefits from a cleaner spatial layout and produces more consistent boundaries.

PCA continues to encode global appearance reliably, and the interaction between the projection and the clustering algorithms becomes more balanced. Ward linkage, in particular, shows improved alignment with the structure induced by PCA and achieves metric values close to or slightly above those of K-means.

Although both clustering methods produced very comparable accuracy on the ground-truth data, the linkage clustering has a higher ability to preserve coherent boundaries when the class distribution is uniform and the geometric structure is well defined.


### 4.2 Non-negative Matrix Factorization (NMF)

NMF decomposes each image into additive, non-negative components that tend to highlight localized visual patterns such as color regions or texture fragments, offering a parts-based representation that differs substantially from the variance-oriented structure produced by PCA.

After projecting the dataset into this feature space, we apply both clustering variants used throughout the analysis, allowing us to evaluate how the NMF decomposition influences the formation of groups under the same experimental conditions.

This parallel treatment provides a direct comparison with the PCA-based models and helps determine whether the localized, interpretable features extracted by NMF lead to improvements in semantic organization across the dataset.

In [ ]:
# ---------------------------------------------------------
# Load labeled dataset
# ---------------------------------------------------------
X = images
y_true = labelled_images["category"]

# Flatten images to 1D vectors
X_flat = X.reshape(len(X), -1)

# Scale to [0,1] for NMF stability
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_flat)

# Encode string labels into integers
le = LabelEncoder()
y_true_int = le.fit_transform(y_true)

# ---------------------------------------------------------
# NMF decomposition
# ---------------------------------------------------------
n_components = 100

print(f"Fitting NMF ({n_components} components)...")
nmf = None
nmf = NMF(n_components=n_components, init='nndsvda', random_state=4, max_iter=100, verbose = True)
X_nmf = nmf.fit_transform(X_scaled)

# Clean any residual NaN or inf (should not appear, but safe)
X_nmf = np.nan_to_num(X_nmf, nan=0.0, posinf=1.0, neginf=0.0)
X_nmf = X_nmf.astype(np.float64)

print("NMF done. Shape of reduced data:", X_nmf.shape)

![NMF](images/nmf.png)

#### Hyperparameters tuning

Different configurations of n_components, tol, random_state, and initialization methods were tested. The model proved highly sensitive to these parameters, with the most stable and consistent results obtained using 100 components, init='nndsvda', and random_seed = 4.



#### Results of NMF K-Means vs Linkage (Ward)

| Metric                              | NMF K-means | NMF Linkage (Ward) |
| ----------------------------------- | ----------- | ------------------ |
| Adjusted Rand Index (ARI)           | 0.036       | 0.726              |
| Normalized Mutual Information (NMI) | 0.132       | 0.664              |
| Homogeneity                         | 0.098       | 0.663              |
| Completeness                        | 0.204       | 0.664              |
| V-Measure                           | 0.132       | 0.664              |
| Optimized Accuracy (Hungarian)      | 0.430       | 0.901              |


Despite relying on a parts-based decomposition, the behaviour of the two clustering methods diverges sharply. K-Means shows limited agreement with the ground truth and highlights the difficulty of separating the NMF components using centroid-based criteria. The clusters remain diffuse and only loosely connected to the semantic structure of the dataset.

Ward linkage, however, responds very differently once the NMF components are standardised. The hierarchical procedure identifies three compact regions with a level of consistency that is unusually strong for an unsupervised model of this type. All metrics rise substantially, and the optimized accuracy approaches values that are close to supervised performance.

Overall, NMF provides a coherent latent space only when combined with scaling and hierarchical clustering. In this configuration, the results are unexpectedly strong and indicate that Ward linkage is able to capture the underlying structure with a clarity that K-Means does not achieve.


### 4.3 Convolutional Autoencoder (PyTorch Implementation)

We will implement this method using PyTorch Convolutional Autoencoder.

This method differentiates from the previous ones by introducing a non-linear learning process.<br>
The architecture consists of an encoder made of convolutional and max-pooling layers that progressively reduce the spatial resolution of the input images, and a decoder using transposed convolutions to reconstruct the original data from a compact latent representation. The network is trained in a fully unsupervised way using a reconstruction loss (MSE), optimized with the Adam algorithm and accelerated through CUDA.

The main operations that will be performed are the following:

- load the dataset through a custom Dataset class and process it with a DataLoader for batched training, resizing each image to 256×256 pixels and normalizing it to the [0,1] range;

- define a convolutional autoencoder composed of an encoder with convolutional and max-pooling layers, and a decoder with transposed convolutions for image reconstruction;

- train the model using the Mean Squared Error (MSE) loss and the Adam optimizer until reconstruction stability is achieved;

- extract the latent feature vectors from the encoder and apply KMeans clustering to evaluate how well the learned representations capture the intrinsic structure of the dataset.

We can expect an improvement in clustering performance, with the Adjusted Rand Index increasing by approximately 0.05–0.10 and the overall clustering accuracy reaching around 80–85%. The model is expected to learn a more coherent latent representation, enabling a clearer separation of semantic categories and a more accurate reconstruction of the underlying visual structures within the dataset.

In [ ]:
le = LabelEncoder()

VERSION = 'v6'
device = 'cuda'
SEED = 4
np.random.seed(SEED)
torch.manual_seed(SEED)

images, labelled_images = load_dataset("combined_balanced_dataset_v6.npz")


# --- Filter and stack only valid images ---
valid_images = []
for img in images:
    arr = np.array(img)
    if arr.shape == (256, 256, 3):
        valid_images.append(arr)
    else:
        print(f"Skipping image with shape {arr.shape}")

images = np.stack(valid_images).astype(np.uint8)
print("Final shape:", images.shape)

# Ensure 'images' is a clean numeric array
if isinstance(images, list) or images.dtype == object:
    # Convert each sub-array to np.uint8 and stack them
    images = np.stack([np.array(img, dtype=np.uint8) for img in images])

print("Images shape:", images.shape, "dtype:", images.dtype)


# Normalize and convert to PyTorch tensors
X_tensor = torch.tensor(images / 255.0, dtype=torch.float32).permute(0, 3, 1, 2)

# Dummy labels (autoencoder is unsupervised)
y_dummy = torch.zeros(len(X_tensor))

# Create dataset and dataloader
dataset = TensorDataset(X_tensor, y_dummy)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)


# ============================================================
# 3. Define Autoencoder model
# ============================================================
class ConvAutoencoder(nn.Module):
    def __init__(self, latent_dim=256):
        super().__init__()
        # --- Encoder ---
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),   # 128x128
            nn.ReLU(True),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),  # 64x64
            nn.ReLU(True),
            nn.Conv2d(64, 128, 3, stride=2, padding=1), # 32x32
            nn.ReLU(True),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),# 16x16
            nn.ReLU(True),
            nn.Flatten(),
            nn.Linear(256*16*16, latent_dim)
        )
        # --- Decoder ---
        self.decoder_fc = nn.Linear(latent_dim, 256*16*16)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1), # 32x32
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),  # 64x64
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),   # 128x128
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 3, 3, stride=2, padding=1, output_padding=1),    # 256x256
            nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(self.decoder_fc(z).view(-1, 256, 16, 16))
        return x_hat, z


# Parameters (Setup A)
max_epochs = 20
patience = 20
tolerance_decimal = 3
latent_dim = 256
lr = 0.001
batch_size =  8


loss_history = []
stable_count = 0
last_epoch = -1
last_loss = -1



# Model, loss, optimizer
model = ConvAutoencoder(latent_dim=latent_dim).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# DataLoader (shuffle must stay False to preserve order)
dataset = TensorDataset(X_tensor, y_dummy)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# ============================================================
# 5. Training loop with early stopping (4th decimal stability)
# ============================================================
for epoch in range(max_epochs):
    model.train()
    running_loss = 0.0

    for imgs, _ in tqdm(dataloader, desc=f"Epoch {epoch+1}/{max_epochs}"):
        imgs = imgs.to(device)
        optimizer.zero_grad()
        recon, _ = model(imgs)
        loss = criterion(recon, imgs)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    loss_history.append(avg_loss)

    print(f"Epoch {epoch+1}/{max_epochs} - AvgLoss: {avg_loss:.6f}")

    # Early stopping check (loss stable within 4 decimals)
    if len(loss_history) > 1:
        prev_loss = loss_history[-2]
        if round(prev_loss, tolerance_decimal) == round(avg_loss, tolerance_decimal):
            stable_count += 1
        else:
            stable_count = 0
        if stable_count >= patience:
            print(f"\nEarly stopping triggered at epoch {epoch+1} "
                  f"(loss stable within 4 decimals for {patience} epochs).")
            break

# ============================================================
# 6. Save model
# ============================================================

#torch.save(model.state_dict(), f"autoencoder_trained_from_images_{VERSION}_seed{SEED}_batch{batch_size}_epoch{max_epochs}_patience{patience}.pt")
#print(f"✅ Model saved -> autoencoder_trained_from_images_{VERSION}_seed{SEED}_batch{batch_size}_epoch{max_epochs}_patience{patience}.pt")

# ============================================================
# 7. Extract embeddings, compute PCA (optional), KMeans and Linkage
# ============================================================

# Switch to eval mode
model.eval()

# Encode all images
all_embeddings = []
with torch.no_grad():
    for imgs, _ in DataLoader(dataset, batch_size=batch_size, shuffle=False):
        imgs = imgs.to(device)
        _, z = model(imgs)
        all_embeddings.append(z.cpu().numpy())

X_autoencoder = np.vstack(all_embeddings)  # shape: (N, latent_dim)

# -------------------------
# PCA on latent space (optional)
# -------------------------
from sklearn.decomposition import PCA
pca = PCA(n_components=50, random_state=4)
X_PCA_autoencoder = pca.fit_transform(X_autoencoder)

# -------------------------
# KMeans
# -------------------------
from sklearn.cluster import KMeans
k = 3
kmeans = KMeans(n_clusters=k, random_state=4)
labels_kmeans_autoencoder = kmeans.fit_predict(X_autoencoder)

# -------------------------
# Linkage (Ward)
# -------------------------
from scipy.cluster.hierarchy import linkage, fcluster

Z_autoencoder = linkage(X_autoencoder, method="ward", metric="euclidean")
labels_linkage_autoencoder = fcluster(Z_autoencoder, t=3, criterion="maxclust")

In [ ]:
# ============================================================
# 7. Extract embeddings, compute PCA (optional), KMeans and Linkage
# ============================================================

# Switch to eval mode
model = ConvAutoencoder(latent_dim=256).to(device)
checkpoint = torch.load("autoencoder_trained_from_images_v6_seed4_batch8_epoch20_patience20-656_667_877_2.pt", map_location=device)
model.load_state_dict(checkpoint)
model.eval()

# Encode all images
all_embeddings = []
with torch.no_grad():
    for imgs, _ in DataLoader(dataset, batch_size=batch_size, shuffle=False):
        imgs = imgs.to(device)
        _, z = model(imgs)
        all_embeddings.append(z.cpu().numpy())

X_autoencoder = np.vstack(all_embeddings)  # shape: (N, latent_dim)

# -------------------------
# PCA on latent space (optional)
# -------------------------
from sklearn.decomposition import PCA
pca = PCA(n_components=50, random_state=4)
X_PCA_autoencoder = pca.fit_transform(X_autoencoder)

# -------------------------
# KMeans
# -------------------------
from sklearn.cluster import KMeans
k = 3
kmeans = KMeans(n_clusters=k, random_state=4)
labels_kmeans_autoencoder = kmeans.fit_predict(X_autoencoder)

# -------------------------
# Linkage (Ward)
# -------------------------
from scipy.cluster.hierarchy import linkage, fcluster

Z_autoencoder = linkage(X_autoencoder, method="ward", metric="euclidean")
labels_linkage_autoencoder = fcluster(Z_autoencoder, t=3, criterion="maxclust")

ari, nmi, acc = show_all_metrics ("Autoencoder", labels_kmeans_autoencoder, labels_linkage_autoencoder, X_autoencoder, Z_autoencoder, 50, k, y_true, le, images, labelled_images)
suffix = str(acc)[2:][:3]


![Autoencoder](images/autoencoder.png)

#### Autoencoder Results
After an extensive series of tests combining seed, batch size, number of epochs, and patience, these metrics represent the best configuration obtained so far:
- seed: 4
- batch size: 8
- max-epochs: 20
- patience: 20

The autoencoder produced a latent space that captures the dominant semantic structure of the dataset. Both clustering methods perform well on this representation, with linkage showing a stronger ability to separate the three visual groups. 

The results indicate that this configuration achieves a balanced embedding that supports stable, unsupervised discrimination of the classes.

| Metric                              | AE K-Means | AE Linkage (Ward) |
| ----------------------------------- | ---------- | ----------------- |
| Adjusted Rand Index (ARI)           | 0.517      | 0.657             |
| Normalized Mutual Information (NMI) | 0.515      | 0.667             |
| Homogeneity                         | 0.507      | 0.660             |
| Completeness                        | 0.523      | 0.675             |
| V-Measure                           | 0.515      | 0.667             |
| Optimized Accuracy (Hungarian)      | 0.804      | 0.877             |


### 4.4 Deep Embedded Clustering (DEC)

Deep Embedded Clustering is an unsupervised method that integrates non-linear representation learning with a dedicated clustering objective. 

The process begins by training a convolutional autoencoder to construct a compact latent representation that preserves the essential visual structure of the input images. 
Once a stable reconstruction space is obtained, the decoder is removed and the encoder is further optimised using a clustering-driven loss.

DEC initialises cluster centroids using KMeans in the latent space, then refines both the embeddings and the cluster assignments through an iterative procedure based on the Student’s t-distribution. This mechanism sharpens cluster boundaries by increasing the contrast between high-confidence assignments and ambiguous samples, gradually transforming the latent space into a more discriminative representation.

This approach is particularly suitable for datasets such as the present one, where class separation depends on subtle shape regularities rather than simple colour statistics. The combination of non-linear feature extraction and clustering-oriented fine-tuning allows DEC to capture spatial structures that linear methods or purely reconstruction-based encoders tend to attenuate. As a result, DEC is well positioned to uncover meaningful semantic partitions even in moderately complex visual domains.


In [ ]:
# ============================================================
# DEEP EMBEDDED CLUSTERING (DEC) - Seed 4 Fine-Tuning
# ============================================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tqdm import tqdm
import joblib


# ------------------------------------------------------------
# 1. Configuration
# ------------------------------------------------------------
images, labelled_images = load_dataset("combined_balanced_dataset_v6.npz")

VERSION = 6
SEED = 4
BATCH_SIZE = 4

np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Running DEC on device: {device} (seed={SEED})")


# ------------------------------------------------------------
# 2. Load hybrid dataset
# ------------------------------------------------------------
X = labelled_images["X"]
y_true = labelled_images["category"]
print("Dataset:", X.shape)

# Normalize to [0,1] and permute
class ImageDataset(Dataset):
    def __init__(self, X):
        self.X = X
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img = self.X[idx]                                # numpy uint8 [H,W,3]
        img = torch.tensor(img, dtype=torch.float32)     # float32
        img = img.permute(2,0,1) / 255.0                 # to [0,1], CHW
        return img
        

dataset = ImageDataset(X)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# ------------------------------------------------------------
# 3. Encoder definition (must match previous architecture)
# ------------------------------------------------------------
class ConvEncoder(nn.Module):
    def __init__(self, latent_dim=256):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256 * 16 * 16, latent_dim)
        )

    def forward(self, x):
        return self.encoder(x)

# Load pretrained encoder
encoder = ConvEncoder().to(device)
checkpoint_full = torch.load("autoencoder_trained_from_images_v6_seed4_batch8_epoch20_patience20-656_667_877_2.pt", map_location=device)

# Adapt keys if they do not have "encoder." prefix
state_dict = {}
for k, v in checkpoint_full.items():
    if k.startswith("encoder."):
        new_key = k.replace("encoder.", "")
        state_dict[new_key] = v
    elif any(s in k for s in ["0.weight", "0.bias", "2.weight", "2.bias"]):
        # direct weights from encoder (no prefix)
        state_dict[k] = v

encoder.load_state_dict(state_dict, strict=False)
encoder.eval()
print("Pretrained encoder weights loaded successfully (seed 4).")



In [ ]:
# ============================================================
# 6. DEC Training Loop (Android-Safe)
# ============================================================


def soft_assign(z, centroids, alpha=1):
    """Compute soft assignment q_ij based on Student t-distribution."""
    dist = torch.cdist(z, centroids, p=2) ** 2
    numerator = (1.0 + dist / alpha) ** (-(alpha + 1) / 2)
    q = numerator / torch.sum(numerator, dim=1, keepdim=True)
    return q

def target_distribution(q):
    """Sharpen the soft assignments to get target distribution p."""
    weight = q ** 2 / torch.sum(q, dim=0)
    p = (weight.t() / torch.sum(weight, dim=1)).t()
    return p

def metrics_on_the_fly(checkpoint):
    # Remap keys: "0.weight" → "encoder.0.weight"
    remapped = {}
    for k, v in checkpoint.items():
        new_key = f"encoder.{k}"
        remapped[new_key] = v

    encoder = ConvEncoder().to(device)
    encoder.load_state_dict(remapped, strict=False)
    encoder.eval()

    features_dec = []
    with torch.no_grad():
        for imgs in tqdm(dataloader, desc="Extracting final latent features"):
            imgs = imgs.to(device)
            z = encoder(imgs)
            features_dec.append(z.cpu())
    features_dec = torch.cat(features_dec).numpy()

    scaler = StandardScaler()
    features_dec = scaler.fit_transform(features_dec)
    kmeans_refined = KMeans(n_clusters=n_clusters, random_state=SEED, n_init=1)
    #labels_kmeans_dec = kmeans_refined.fit_predict(features_dec)

    # DEC LINKAGE

    # 1. Usa già features_dec normalizzate e scalate
    X_dec = features_dec  # shape [N, latent_dim] dopo StandardScaler

    # 2. Linkage Ward
    Z_linkage_dec = linkage(X_dec, method="ward")

    # 3. Cut-tree per ottenere k cluster
    labels_linkage_dec = fcluster(Z_linkage_dec, t=3, criterion="maxclust")

    # 4. Valutazione
    ari = adjusted_rand_score(y_true, labels_linkage_dec)
    nmi = normalized_mutual_info_score(y_true, labels_linkage_dec)
    v = v_measure_score(y_true, labels_linkage_dec)

    print(f"DEC + Linkage Ward | ARI={ari:.4f} | NMI={nmi:.4f} | V={v:.4f}")
    return ari,nmi,v

SEED = 4
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#device = "cuda"
device = "cpu"

MAX_EPOCHS =200
PATIENCE = 100
DELTA = 1e-4
ALPHA = 1  # Student t-distribution parameter

encoder = ConvEncoder().to(device)
checkpoint = torch.load(f"AE_undertrained_seed{SEED}_{device}_v{VERSION}.pt", map_location=device)

# Remap keys: "0.weight" → "encoder.0.weight"
remapped = {}
for k, v in checkpoint.items():
    new_key = f"encoder.{k}"
    remapped[new_key] = v

encoder = ConvEncoder().to(device)
encoder.load_state_dict(remapped, strict=False)
encoder.eval()

print("Undertrained encoder loaded successfully and mapped to ConvEncoder.")

encoder.eval()
print("Pretrained encoder weights loaded successfully (seed 4).")


initial_centroids = torch.tensor(kmeans_init.cluster_centers_, dtype=torch.float32).to(device)

optimizer = optim.Adam(encoder.parameters(), lr=1e-4)
criterion = nn.KLDivLoss(reduction='batchmean')

best_loss = float("inf")
patience_counter = 0
last_epoch = 0

import copy

best_ari = -1
best_nmi = -1
best_state = None
best_epoch = -1

prev_loss = None
loss_descent = 0

for epoch in range(MAX_EPOCHS):
    # ----------------------------------------------
    # STEP 1: Extract all latent vectors (z)
    # ----------------------------------------------
    encoder.eval()
    with torch.no_grad():
        z_full = []
        for imgs in dataloader:
            imgs = imgs.to(device)
            z_full.append(encoder(imgs))
        z_full = torch.cat(z_full)  # shape [N, latent_dim]

    # ----------------------------------------------
    # STEP 2: Compute q and p over full dataset
    # ----------------------------------------------
    q = soft_assign(z_full, initial_centroids)
    p = target_distribution(q)  # shape [N, k]

    # ----------------------------------------------
    # STEP 3: Update centroids (critical fix)
    # ----------------------------------------------
    with torch.no_grad():
        numer = torch.matmul(q.t(), z_full)         # shape [k, latent_dim]
        denom = q.sum(dim=0, keepdim=True).t()      # shape [k, 1]
        updated_centroids = numer / denom           # shape [k, latent_dim]
        initial_centroids = updated_centroids       # overwrite

    # ----------------------------------------------
    # STEP 4: Train encoder using mini-batches
    # ----------------------------------------------
    encoder.train()
    epoch_loss = 0.0
    
    for batch_idx, imgs in enumerate(dataloader):
        imgs = imgs.to(device)
        z = encoder(imgs)
        q_batch = soft_assign(z, initial_centroids)
        
        start = batch_idx * imgs.size(0)
        end = start + imgs.size(0)
        p_batch = p[start:end]

        loss = criterion(torch.log(q_batch + 1e-8), p_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)
    if prev_loss is None:
        prev_loss = 1000
    print(f"Epoch {epoch+1}/{MAX_EPOCHS} - KL Loss: {avg_loss:.6f}")

    # ----------------------------------------------
    # Optional evaluation (unchanged)
    # ----------------------------------------------
    encoder.eval()
    with torch.no_grad():
        z_eval = []
        for imgs in dataloader:
            imgs = imgs.to(device)
            z_eval.append(encoder(imgs))
        z_eval = torch.cat(z_eval).cpu().numpy()

    ari_t,nmi_t,v_t = metrics_on_the_fly(encoder.state_dict())


    print(f"                                  [Progress @Epoch {epoch+1}] ARI={ari_t:.4f} | NMI={nmi_t:.4f} | V-Measure={v_t}" )
    # Track best model based on ARI (or NMI)
    if prev_loss > avg_loss:
        loss_descent += 1
        print(f"                                                       DESCENT: {loss_descent}")

#    if ari_t > best_ari:  # or use nmi_t if preferred
    best_ari = ari_t
    best_nmi = nmi_t
    best_epoch = epoch + 1
    best_state = copy.deepcopy(encoder.state_dict())
    print(f"                                                       >>> NEW BEST (epoch {best_epoch}) ARI={best_ari:.4f} NMI={best_nmi:.4f}")
    torch.save(best_state, f"DEC_encoder_best_seed{SEED}_{device}_{VERSION}_bestepoch_{best_epoch}_{best_ari:.3f}_{best_nmi:.3f}_alpha{ALPHA}_batch{BATCH_SIZE}.pt")
    print(f"                                                           saved @ epoch {best_epoch} -> DEC_encoder_best_seed{SEED}_{device}_{VERSION}_bestepoch_{best_epoch}_{best_ari:.3f}_{best_nmi:.3f}_alpha{ALPHA}_batch{BATCH_SIZE}.pt")


    # ----------------------------------------------
    # Early stopping
    # ----------------------------------------------
    if avg_loss < best_loss:
        best_loss = avg_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"Early stopping at epoch {epoch+1}")
            break

    last_epoch = epoch
    prev_loss = avg_loss
    
    




![DEC](images/dec.png)

#### DEC Results

| Metric                              | DEC K-Means | DEC Linkage (Ward) |
| ----------------------------------- | ----------- | ------------------ |
| Adjusted Rand Index (ARI)           | 0.440       | 0.684              |
| Normalized Mutual Information (NMI) | 0.471       | 0.645              |
| Homogeneity                         | 0.460       | 0.644              |
| Completeness                        | 0.482       | 0.646              |
| V-Measure                           | 0.471       | 0.645              |
| Optimized Accuracy (Hungarian)      | 0.643       | 0.883              |


As we can see from the above metrics, DEC with K-Means clustering does not provide a meaningful improvement, which is not surprising given that in this particular scenario the RGB images are not easily separable by simple centroid-based boundaries. 
Instead, <b>the linkage-based approach delivers a clear performance gain<b>, reaching ARI 0.684, NMI 0.645 and an optimized <b>accuracy of 0.883</b>, the highest of all the methods analyzed. 

This indicates that the latent space learned by DEC does contain discriminative structure, but its topology is better captured by hierarchical relationships than by fixed-radius partitions. 

We can conclude that when the visual differences between classes are subtle and distributed, like in the dataset of images of this work, the linkage clustering appears to exploit the embedding more effectively, producing a separation that is both robust and semantically coherent.


### 4.5 Supervised baseline (SVC classifier)

Finally, we introduce a supervised SVC model to estimate the maximum accuracy that can be realistically achieved on this dataset. 

As the title suggests, this experiment is included purely as a reference point, allowing us to assess the intrinsic limitations of unsupervised learning and quantify the gap between our best clustering results and a label-driven approach. 

A Support Vector Classifier is particularly suitable in this setting, as it performs well in high-dimensional continuous feature spaces and has a long record of effectiveness on image data transformed through latent representations or dimensionality reduction. 

The aim is not to optimize the supervised model, but to establish a credible upper bound that validates the dataset’s separability and frames the performance of the unsupervised techniques in a meaningful way.


In [ ]:
# ============================================================
# Supervised Baseline – SVC Classifier (3 Categories)
# ============================================================

# ------------------------------------------------------------
# 1. Extract features and labels
# ------------------------------------------------------------
X = labelled_images["X"]               # shape (N, features)
# Flatten to (N, features)
X = X.reshape(len(X), -1)

y = labelled_images["category"]

# Encode categories into integers 0/1/2
le = LabelEncoder()
y_int = le.fit_transform(y)

# ------------------------------------------------------------
# 2. Train/test split
# ------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y_int, test_size=0.25, random_state=42, stratify=y_int
)

# ------------------------------------------------------------
# 3. Train SVC
# ------------------------------------------------------------
clf = SVC(kernel="rbf", C=2, gamma="scale")
clf.fit(X_train, y_train)

# ------------------------------------------------------------
# 4. Evaluation
# ------------------------------------------------------------
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Supervised SVC Accuracy: {acc:.3f}\n")

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Supervised SVC Accuracy: 0.942

Classification Report:
              precision    recall  f1-score   support

     animals       0.93      0.89      0.91        28
       birds       0.96      0.93      0.95        29
   landscape       0.94      1.00      0.97        29

    accuracy                           0.94        86
   macro avg       0.94      0.94      0.94        86
weighted avg       0.94      0.94      0.94        86



#### supervised SVC results

Supervised SVC Accuracy: 0.942
<pre>
Classification Report:
                precision    recall  f1-score   support

animals         0.93      0.89      0.91        28
birds           0.96      0.93      0.95        29
landscape       0.94      1.00      0.97        29

accuracy                            0.94        86
macro avg       0.94      0.94      0.94        86
weighted avg    0.94      0.94      0.94        86
</pre>
A brief comparison with the supervised baseline confirms that the dataset is internally consistent and sufficiently informative, as a simple SVC reaches an accuracy of approximately 0.94 without any specialized optimization.

The lower performance of the unsupervised models was expected, since they operate without labels and rely purely on latent structure.

However, the relatively small gap of about six percentage points between the best DEC + linkage configuration (~0.88) and the supervised reference (~0.94) shows that the unsupervised approach is far from ineffective. 

On the contrary, it demonstrates a meaningful capacity to recover semantic groupings directly from the data, without relying on annotated examples, and therefore remains a viable option when labels are costly or unavailable.

## Conclusions

In this project we explored a hierarchy of unsupervised representation and clustering strategies on an RGB image dataset with three semantic categories. Starting from basic linear decompositions, and gradually moving toward more advanced neural approaches, we built a clear trajectory: **PCA → NMF → Autoencoder → DEC Fine-Tuned (RGB)**. 

All techniques were applied using only the raw RGB channels as input, and the comparison across K-Means and linkage clustering revealed how different methods extract structure from the same visual signal, highlighting the relationship between latent representation quality, clustering behavior, and computational efficiency.

The following table shows the metrics collected during this work:

| Model                   | Configuration                               | Accuracy (%) | ARI   | NMI   | V-Measure | Notes                                                       |
| ----------------------- | ------------------------------------------- | ------------ | ----- | ----- | --------- | ----------------------------------------------------------- |
| PCA + K-Means           | n_components = 200                          | 79.5         | 0.490 | 0.487 | 0.487     | Solid baseline, acceptable performance                      |
| PCA + Linkage           | n_components = 200                          | 79.8         | 0.525 | 0.539 | 0.539     | Slight improvement over K-Means                             |
| NMF + K-Means           | n_components=100, max_iter=100, normalized  | 43.0         | 0.036 | 0.132 | 0.132     | Very poor separation with K-Means                           |
| NMF + Linkage           | n_components=100, max_iter=100, normalized  | <b>90.1</b>  | 0.726 | 0.664 | 0.664     | <b>Highest overall metrics among all models</b>             |
| Autoencoder + K-Means   | epochs=20, patience=20, batch=8             | 60.8         | 0.387 | 0.412 | 0.412     | Moderate results, clearly better than NMF                   |
| Autoencoder + Linkage   | epochs=20, patience=20, batch=8             | 87.7         | 0.657 | 0.667 | 0.667     | Strong performance, close to supervised                     |
| DEC + K-Means           | max_epoch=110, patience=30, alpha=1.0       | 64.3         | 0.440 | 0.471 | 0.471     | Better than AE+KMeans, weaker than linkage                  |
| DEC + Linkage           | max_epoch=110, patience=30, alpha=1.0       | 88.3         | 0.684 | 0.645 | 0.645     | Best model for mobile and GPU implementation                |
| Supervised SVC          | C=2, test_size=0.25                         | 94.2         |       |       |           | Precision 0.93–0.96, Recall 0.89–1.00, F1 up to 0.97        |


In summary, the results confirm that even when restricted to raw RGB information, unsupervised learning can recover a meaningful semantic structure from image data. 

The progression from PCA to NMF, to neural embeddings with autoencoder, and finally to DEC illustrates how increasingly expressive representations affect cluster separability, with linkage-based methods generally outperforming K-Means across all models.

The results also show that, with targeted tuning and careful selection of parameters and hyperparameters, each method can achieve meaningful performance, with accuracy values ranging approximately from 79% to over 90% depending on the model and clustering strategy.

<b>NMF combined with hierarchical clustering achieved the best overall metrics</b>, demonstrating that simplicity and interpretability do not necessarily limit effectiveness. 

Neural approaches, while more costly to train, offered competitive results and showed particular robustness when using linkage clustering, narrowing the gap with the supervised reference model.

From an operational perspective, the choice of model also depends on deployment constraints. While NMF remains the most accurate in absolute terms, DEC provides a compact neural encoder compatible with PyTorch Mobile and GPU execution, making it the most practical option for an Android demonstration where lightweight inference and library support are essential.

## Android Implementation Demo

An Android demo app is being developed to demonstrate the practical potential of the DEC-based unsupervised training.

The application uses the device’s camera to perform **real-time inference**, detecting whether the object or scene currently in the frame corresponds to one of the three discovered clusters: 
- *animal*,
- *bird*, 
- or *landscape*.

This lightweight mobile implementation highlights how an unsupervised model, once trained, can be deployed efficiently on edge devices without the need for labeled data or cloud computation.

You can see the demo in action at the following link:
https://www.youtube.com/watch?v=fvw-nV2-CKw